# Mastering with ROS: TIAGo

<img src="img/pal-logo.png" width="400" />

<img src="img/opencv_tiago_face1.png" width="600"/>

<img src="img/robotignite_logo_text.png" width="400"/>

## Unit 4: Perception with OpenCV

<p style="background:green;color:white;">SUMMARY</p>

Estimated time of completion: **2h**

Perform various Perception tasks using the OpenCV library.

<p style="background:green;color:white;">END OF SUMMARY</p>

OpenCV (Open Source Computer Vision Library) is an open source computer vision and machine learning software library. OpenCV was built to provide a common infrastructure for computer vision applications and to accelerate the use of machine perception in the commercial products. Being a BSD-licensed product, OpenCV makes it easy for businesses to utilize and modify the code.
You can check more information about on the following link: https://opencv.org/

In the following Unit you are going to see different examples of Perception based on the RGB data that TIAGo camera provides using the OpenCV library. Basically, you are going to see the following:

* Corner Detection
* Find Keypoints
* Matching
* ArUco marker detection
* Person Detection
* Face Detection
* Planar object detection and pose estimation

In the simulation for this Unit simulation, TIAGo robot has spawned in front of the stairs, which will allows us to start performing Corner Detection.

## Corner Detection

In this section you are going to see how to use the <a href="https://docs.opencv.org/2.4.8/modules/imgproc/doc/feature_detection.html?#goodfeaturestotrack" target="_blank">Shi-Tomasi</a> and <a href="https://docs.opencv.org/2.4.8/modules/imgproc/doc/feature_detection.html?#cornerharris" target="_blank">Harris</a> corner detection functions changing two simple parameters.

<p style="background:#EE9023;color:white;">Exercise 4.1</p>

a) Execute the following command in order to run the Corner Detection node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun tiago_opencv_tutorial corner_detection

Now, in the Graphical Tools tab, two windows will show up, one with the corners detected by the Harris function and one with the Shi-Tomasi one.

<img src="img/opencv_corners.png" width="700" />

<p style="background:#EE9023;color:white;">End of Exercise 4.1</p>

### Code

The difference in the Shi-Tomasi and Harris functions is slight, however it becomes more apparent when their equation is shown.

Firstly, a corner in the OpenCV Mat structure is defined as a region in which the intensity varies greatly in all directions. The most important feature in the two methods is the scoring function R. The Harris method defines the scoring function as

In [ ]:
R = v * w - k(v + w)^2

as represented in the source code as

In [ ]:
for( int j = 0; j < img_gray.rows; j++ )
  for( int i = 0; i < img_gray.cols; i++ )
  {
    float lambda_1 = myHarris_dst.at<cv::Vec6f>(j, i)[0];
    float lambda_2 = myHarris_dst.at<cv::Vec6f>(j, i)[1];
    Mc.at<float>(j,i) = lambda_1*lambda_2 - 0.04f*pow( ( lambda_1 + lambda_2 ), 2 );
  }

where v and w are the Eigen values of the matrix containing the image. In this case the difference in t Shi-Tomasi proposed a different cost function, namely

In [ ]:
R = min(v,w)}

which simply states that if the eigen values are higher than a certain threshold value, the region is considered to be a corner. The resulting corners detected are slightly better, and more suited for some applications such as object tracking.

## Find Keypoints

In order to track objects more effectively, or against a dynamic background, a method is to detect keypoints in a static image of the target object, create a keypoint profile, and match this against a feed of images. In this tutorial the keypoint detectors available in OpenCV 2.4 will be shown, along with some simple image manipulation in the form of contrast and sharpness.

<p style="background:#EE9023;color:white;">Exercise 4.2</p>

a) First of all, you will need to move the TIAGo robot in front of the REEM poster that is in the simulation. Yes! I know what you are thinking... there's no poster in the simulation! Well, apparently there isn't, but that's not totally true. Due to some limitations in Gzweb, you are not able to visualize it correctly, but the poster is there. In fact, when you visualize through TIAGos camera, you will be able to see it properly. So, for now, just move the robot to this position.

<img src="img/opencv_tiago_poster.png" width="600" />

b) Execute the following command in order to start the keypooint extractor node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch tiago_opencv_tutorial keypoint_tutorial.launch

In the Graphic Tools tab, you will see something like this:

<img src="img/opencv_keypoint1.png" width="600" />

This GUI allows you to manipulate the types of detectors, sharpness and brightness of the image.

The gui is divided into 4 parts. The Kernel Matrix for Sharpening Image allows the user to manipulate the values present in the kernel used by the OpenCV filter2D() method.

The contrast parameters influence the alpha and beta values used by the saturate_cast<uchar>() method.

As there are multiple variables affected in this tutorial, several windows have been created to allow the user to see the effect of the parameters changed individually and as a whole. The CV Windows section allows the user to open and close certain windows.

Detector Algorithms is the selection list of the keypoint detector used in this tutorial. The list contains the following entries:

* SURF
* SIFT
* ORB
* BRISK
* MSER
* GFTT
* Harris
* Dense
* SimpleBlob
* FAST
* STAR

Also, another window is launched where the image processed with the keypoints detected overlaid.

<img src="img/opencv_keypoint2.png" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 4.2</p>

### Contrast

The contrast change makes simple iterative use of the OpenCV saturate_cast<uchar>() method.

In [ ]:
for(int i = 0; i<in.rows; i++)
  for(int j = 0; j<in.cols; j++)
    for(int k = 0; k<3; ++k)
      out.at<cv::Vec3b>(i,j).val[k] = cv::saturate_cast<uchar>(alpha * (in.at<cv::Vec3b>(i,j).val[k] + beta));

These for loops iterate through every element in the matrix, which in turn contains three values for BGR. These values are altered using the alpha and beta values, causing the intensity to increase or decrease. The saturate reference in the function refers to fact that the method only outputs values within the limits of the target type. If it is without these limits, then the value is set to either the minimum or maximum, depending which limit is exceeded.

### Sharpness

The sharpness of the image is affected by the filter2d() OpenCV method. This takes in the target image and a 3x3 matrix kernel, and convolves the image. The initial kernel is the kernel for sharpening an image, however by changing the kernel the image can be adapted for edge detection, blur and unsharp images.

### Detectors

The detectors used are all part of OpenCV, with the most notable being SURF, SIFT and ORB. These detectors all function in different ways, with ORB being one of the most reputable of detectors. They return a vector of KeyPoints, which are then drawn onto the original image.

## Matching

In the following section you are going to see  the capabilities of different detectors with different matchers.

<p style="background:#EE9023;color:white;">Exercise 4.3</p>

a) Execute the following command in order to start the matching node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch tiago_opencv_tutorial matching_tutorial.launch

In the Graphic Tools tab, you will see something like this:

<img src="img/opencv_matching1.png" width="600" />

This GUI will allow you to select feature detectors, descriptor extractors and descriptor matchers. It also allows the user to set the 'k' and 'distance percentage' variables, which are used in the matcher part of the processes.

Also, another window will show up where the matched keypoints between the reference image and the video live feed are presented.

<img src="img/opencv_matching2.png" width="600" />

The reference image is set by default to

`rospack find tiago_opencv_tutorial`/resources/REEM.png

But it can be set to any image by using the text box on the bottom part of the GUI. The filename of the new image with its absolute path needs to be typed.

It is interesting to move the robot around the world and see how robust are the different feature and descriptor extractor algorithms.

<p style="background:#EE9023;color:white;">End of Exercise 4.3</p>

The image processing is divided into three parts. The first was handled in the previous tutorial, namely feature extraction. The next is descriptor extraction, which extracts the vectors describing the images interesting features. The last is the use of knnMatching for detecting possible matching features.

## ArUco marker detection

In the following section you are going to see how to run fiducial marker detection with TIAGo using the OpenCV-based library ArUco. Fiducial markers is an example of robust model-based object detection. The ArUco markers are planar square binary markers that, provided the length of its square side, its 6 DoF pose can be estimated precisely from monocular vision.

<p style="background:#EE9023;color:white;">Exercise 4.4</p>

a) First of all, you will need to move the TIAGo robot in front of the ArUco board that is in the simulation. Yes! I know what you are thinking... there's no board in the simulation! Well, apparently there isn't, but that's not totally true. Due to some limitations in Gzweb, you are not able to visualize it correctly, but the poster is there. In fact, when you visualize through TIAGos camera, you will be able to see it properly. So, for now, just move the robot to this position.

<img src="img/opencv_tiago_aruco.png" width="600" />

b) Execute the following command in order to start the marker detector node with the appropriate remappings and parameters.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun aruco_ros single /camera_info:=/xtion/rgb/camera_info /image:=/xtion/rgb/image_raw _marker_id:=584  _marker_size:=0.10 _marker_frame:=/aruco_frame _camera_frame:=/xtion_optical_frame _image_is_rectified:=true _reference_frame:=/base_footprint

In the above instruction we are telling the node to identify the marker with ID 584 and that the length of the square side is 10 cm.

b) Now, you can launch RViz and add an Image display. Setting the image topic to **/xtion/rgb/image_raw**, you will be able to visualize the ArUco board as the robot sees it.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rosrun rviz rviz

<img src="img/opencv_aruco_image.png" width="600" />

Now, you can add a TF display to RViz. Among all the TFs that will be printed, you will be able to see one **aruco_frame**, which represents the position of the marker that TIAGo is detecting.

<img src="img/opencv_aruco_frame.png" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 4.4</p>

## Person detection

In the following section you are going to check a ROS node that subscribes to an image topic and applies the OpenCV person detector based on an Adaboost cascade of HoG. The node publishes ROIs of the detections and a debug image showing the processed image with the ROIs likely to contain a person.

<p style="background:#EE9023;color:white;">Exercise 4.5</p>

a) First of all, you will need to move the TIAGo robot in front of one of the persons that appear in the simulation. Something like this:

<img src="img/opencv_tiago_person1.png" width="600" />

b) Execute the following command in order to start the person detector node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch pal_person_detector_opencv detector.launch image:=/xtion/rgb/image_raw

b) Now, you can run the following command in order to visualize the detection with the image_view node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosrun image_view image_view image:=/person_detector/debug

<img src="img/opencv_aruco_image.png" width="600" />

Note that persons detected are stroked with a ROI. The detections are not only represented in the **/person_detector/debug** Image topic but also in **/person_detector/detections**, which contains a vector with the image ROIs of all the detected persons.

c) Execute the following command in order to see the detection in the **/person_detector/detections** topic.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rostopic echo -n 1 /person_detector/detections

<img src="img/opencv_person_topic.png" width="600" />

d) Move around the TIAGo robot and check how the person detector performs.

<div style="text-align: left">
<img src="img/opencv_tiago_person2.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/opencv_person_detect2.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
</div>

<div style="text-align: left">
<img src="img/opencv_tiago_person3.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/opencv_person_detect3.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
</div>

<p style="background:#EE9023;color:white;">End of Exercise 4.5</p>

## Face detection

In the following section you are going to check a ROS node that subscribes to an image topic and applies the OpenCV face detector based on an Adaboost cascade of Haar features. The node publishes ROIs of the detections and a debug image showing the processed image with the ROIs likely to contain faces.

<p style="background:#EE9023;color:white;">Exercise 4.6</p>

a) First of all, you will need to move the TIAGo robot in front of one of the persons that appear in the simulation, but a little bit closer than in the previous exercise. This way TIAGo will be able to better detect the face of the person. Something like this:

<img src="img/opencv_tiago_face1.png" width="600" />

b) Execute the following command in order to start the face detector node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch pal_face_detector_opencv detector.launch image:=/xtion/rgb/image_raw

b) Now, you can run the following command in order to visualize the detection with the image_view node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosrun image_view image_view image:=/pal_face/debug

<img src="img/opencv_face_detect1.png" width="600" />

Note that faces detected are stroked with a ROI. The detections are not only represented in the **/pal_face/debug** Image topic but also in **/pal_face/faces**, which contains a vector with the image ROIs of all the detected faces.

c) Execute the following command in order to see the detection in the **/pal_face/faces** topic.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rostopic echo -n 1 /pal_face/faces

<img src="img/opencv_face_topic.png" width="600" />

<p style="background:#EE9023;color:white;">End of Exercise 4.6</p>

## Planar object detection and pose estimation 

In the following section you are going to check a ROS node that subscribes to the live video feed of TIAGo and looks for keypoints in order to detect a known planar textured object. When found, the homography between the current view and the reference view is estimated. Then, using the known width and height of the object its 3D pose is also estimated. OpenCV is used in order to extract and match keypoints and to estimate the homography.

<p style="background:#EE9023;color:white;">Exercise 4.7</p>

a) First of all, you will need to move the TIAGo robot in front of one of the textured PAL poster that appears in the simulation. Yes! I know what you are thinking again... there's no PAL poster in the simulation! Well, apparently there isn't, but that's not totally true. Due to some limitations in Gzweb, you are not able to visualize it correctly, but the poster is there. In fact, when you visualize through TIAGos camera, you will be able to see it properly. So, for now, just move the robot to this position:

<img src="img/opencv_tiago_reemc.png" width="600" />

b) Execute the following command in order to start the texture detector node.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch pal_texture_detector texture_detector.launch rectified_image:=/xtion/rgb/image_raw camera_info:=/xtion/rgb/camera_info

The texture detector will be inspecting the live video feed from TIAGo's RGBD camera looking for matches of the image in

In [ ]:
`rospack find pal_texture_detector`/objects/pal_poster.png

which is shown below

<img src="img/reemc_pal.png" width="200" />

b) Now, in the Graphical Tools tab, three debug images will appear. One showing the kepoint matches found between the live video feed and the reference image; one with the current image wrapped according to the homography estimated in order to align it with the reference image; and the keypoint matches found between the wrapped and the reference image in order to validate the homography estimation.

<img src="img/opencv_object1.png" width="600" />

In case that enough matches are found between both images the following topics will be published:

In [ ]:
/texture_detector/debug
/texture_detector/detection
/texture_detector/pose

c) Execute the following command in order to visualize the **/texture_detector/debug** topic. 

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosrun image_view image_view image:=/texture_detector/debug

<img src="img/opencv_object2.png" width="600" />

This topic contains an image with the detected object marked with a green rectangle.

e) The **/texture_detector/detection** topic contains a copy of the image processed and the coordinates of the 4 pixels enclosing the detection.

f) Execute the following command in order to visualize the **/texture_detector/pose** topic. 

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rosrun rviz rviz -d `rospack find pal_texture_detector`/config/rviz.rviz

<img src="img/opencv_object3.png" width="600" />

This is a topic of type **geometry_msgs/PoseStamped**, with the 3D pose of the object estimated by the homography.

<p style="background:#EE9023;color:white;">End of Exercise 4.7</p>

## Congratulations! You now know how to perform Perception using OpenCV with TIAGo!